**Mounting Drives**


In [2]:
from google.colab import files, drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Ridge Regression Analysis**


In [117]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import math

url = '/content/drive/My Drive/input/wx-frontier-agg.csv'
dataframe = pd.read_csv(url, index_col=0)
data = dataframe.values

## I think this is where my errors are coming from; what is train_X and train_Y
X, y = data[:int(math.floor(dataframe.shape[0]*.80)),:], data[int(math.floor(dataframe.shape[0]*.80)):,:]
X = X.transpose()
y = y.transpose()

# Define Ridge Model
ridge_Model = Ridge(alpha=1.0)

# Five-fold cross Validation
cvScoreRidge = cross_val_score(ridge_Model, X, y, scoring="neg_mean_squared_error", cv=5)
cvScoreRidge = np.absolute(cvScoreRidge)
cvScoreRidge = math.sqrt(cvScoreRidge.mean())

## RMSE Scores
print("Ridge Regression RMSE Score {}".format(round(cvScoreRidge, 3)))

Ridge Regression RMSE Score 32.601


**Random Forest**


In [115]:
from sklearn import tree
from sklearn import ensemble

# Define Random Forest Model
RF_model = ensemble.RandomForestRegressor()

# Five Fold Cross Validation
cvScoreRF = cross_val_score(RF_model, X, y, scoring="neg_mean_squared_error", cv=5)
cvScoreRF = np.absolute(cvScoreRF)
cvScoreRF = math.sqrt(cvScoreRF.mean())

## RMSE Scores
print("Random Forest RMSE Score {}".format(round(cvScoreRF, 3)))

Random Forest RMSE Score 608.635
